In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [17]:
import re
from nltk.corpus import stopwords
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import precision_score, recall_score
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument

In [3]:
data=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [4]:
train_data=data[['text','target']]

In [9]:
train_data.isna().sum()

text      0
target    0
dtype: int64

In [15]:
class MyTaggedDocument(object):
    
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
        
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
           
            #Converting All Text to Lower Case
            rawData=doc.lower()
            
            #Removing stopwords
            stop_words = stopwords.words('english')
            stopwords_dict = Counter(stop_words)
            rawData = ' '.join([word for word in doc.split() if word not in stopwords_dict])
            
            yield TaggedDocument(rawData.split(),[self.labels_list[idx]])

In [7]:
train_docs = MyTaggedDocument(train_data['text'].to_list(), train_data['target'].to_list())

In [13]:
train_docs

In [9]:
model = doc2vec.Doc2Vec(vector_size=300, window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025,epochs=10) # use fixed learning rate

In [18]:
model.build_vocab(train_docs)

In [19]:
model.train(train_docs,total_examples=model.corpus_count,epochs=model.epochs)

In [20]:
def vector_for_learning(model,input_docs):
    sents=input_docs
    target,feature_vectors=zip(*[(doc.tags[0],model.infer_vector(doc.words,epochs=20)) for doc in sents])
    return target,feature_vectors

In [21]:
y_train,x_train=vector_for_learning(model,train_docs) 

In [23]:
log_reg= LogisticRegression(random_state=0,n_jobs=1,C=1e5,solver='saga',max_iter=800)

log_reg.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(C=100000.0, max_iter=800, n_jobs=1, random_state=0,
                   solver='saga')

In [24]:
test_data=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [25]:
test_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [36]:
texts=test_data['text'].to_list()
y_test=test_data['target'].to_list()

In [35]:
y_pred=[]
for id, text in enumerate(texts):
    #Creating MyTaggedDocument object for pdf    
    obj=MyTaggedDocument([text],[3])

    #Making numpy arrays to pass the data to classifier
    y,x=vector_for_learning(model,obj)

    #Making predictions on our data
    predicted_labels=log_reg.predict(x)
    y_pred.extend(predicted_labels)

In [37]:
print('Testing Accuracy : %s' % accuracy_score(y_test,y_pred))  
print('Testing F1 score : {}'.format(f1_score(y_test,y_pred,average='weighted')))
print('Precision Score : {}'.format(precision_score(y_test,y_pred,average='weighted')))
print('Recall Score : {}'.format(recall_score(y_test,y_pred,average='weighted')))

Testing Accuracy : 0.7688164980953632
Testing F1 score : 0.7654908652124499
Precision Score : 0.7692778673503539
Recall Score : 0.7688164980953632
